# **COVID 19 Vaccines Up to Date for United States**

Run on Python 3.12.0 | No errors | No warnings

In [1]:
# -----------------------
# Step 0: Import packages
# ----------------------- 

# For data manipulation
import numpy as np
import pandas as pd

# for displaying and modifying the working directory
import os as os

In [2]:
# -----------------------------------------------
# Step 1: Load dataset into dataframe and verify
# -----------------------------------------------

# Tell python where to find the dataset and load it to dataframe df0
os.chdir(r'D:\OneDrive\Documents\Python\Current\COVID 19 Vaccine Up to Date') # absolute path, using \ and r prefix
df0 = pd.read_csv("COVID-19_Vaccines_Up_to_Date_Status.csv")

# Display the first 5 rows of the dataframe
df0.head()

,Date,Location,Demographic_Category,census,Up_to_date,Up_to_date_pct_agegroup
0,10/11/2023,AK,Male_Ages_<5yrs,26153,158,0.6
1,10/11/2023,AL,Female_Ages_2-4_yrs,88242,147,0.2
2,10/11/2023,AR,Female_Ages_50-64_yrs,291030,44168,15.2
3,10/11/2023,AS,Ages_12-17_yrs,5582,0,0.0
4,10/11/2023,AS,Male_Ages_65-74_yrs,1051,0,0.0


In [3]:
# ------------------------------------------------------------
# Step 2: Initial Exploratory Data Analysis (EDA) and Cleaning
# ------------------------------------------------------------

# Display basic information about the data 
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12685 entries, 0 to 12684
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Date                     12685 non-null  object 
 1   Location                 12685 non-null  object 
 2   Demographic_Category     12685 non-null  object 
 3   census                   12685 non-null  int64  
 4   Up_to_date               12685 non-null  int64  
 5   Up_to_date_pct_agegroup  12685 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 594.7+ KB


We're starting with 12,685 rows and 6 columns.

In [4]:
# Display the dataframe shape (rows, columns)
df0.shape

(12685, 6)

In [5]:
# Display descriptive statistics about the data
df0.describe()

,census,Up_to_date,Up_to_date_pct_agegroup
count,1.268500e+04,1.268500e+04,12685.000000
mean,5.374280e+05,8.556774e+04,13.460899
std,1.205460e+06,2.337061e+05,18.006691
min,0.000000e+00,0.000000e+00,0.000000
25%,4.610100e+04,6.900000e+01,0.000000
50%,1.853310e+05,8.551000e+03,5.800000
75%,5.307120e+05,7.157700e+04,18.400000
max,1.986737e+07,4.780943e+06,92.700000


The numeric data looks good. Nothing concerning.

In [6]:
# Display column names 
df0.columns

Index(['Date', 'Location', 'Demographic_Category', 'census', 'Up_to_date',
       'Up_to_date_pct_agegroup'],
      dtype='object')

In [7]:
# Check for missing values
df0.isna().sum()

Date                       0
Location                   0
Demographic_Category       0
census                     0
Up_to_date                 0
Up_to_date_pct_agegroup    0
dtype: int64

There are no missing (NA) values.

In [8]:
# Convert date objects to date values
# df0['Date'] = pd.to_datetime(df0['Date'])

In [9]:
# Drop the 'Up_to_date_pct_agegroup' column as it's not needed. We can calculate it as needed in Tableau.
df0=df0.drop('Up_to_date_pct_agegroup', axis=1)

In [10]:
# Verify the column has been dropped and that we still have the same number of records
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12685 entries, 0 to 12684
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  12685 non-null  object
 1   Location              12685 non-null  object
 2   Demographic_Category  12685 non-null  object
 3   census                12685 non-null  int64 
 4   Up_to_date            12685 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 495.6+ KB


In [11]:
# Drop the demographic categories starting with Age (they're aggregate data and not needed)
df0 = df0.drop(df0[df0['Demographic_Category'].str.startswith('Age')].index)

In [12]:
# Drop the demographic categories starting with Sex (they're aggregate data and not needed)
df0 = df0.drop(df0[df0['Demographic_Category'].str.startswith('Sex')].index)

In [13]:
# Drop the demographic categories we don't need (they overlap with other categories and create duplicate data)
df0 = df0.drop(df0[df0['Demographic_Category'].str.endswith('<5yrs')].index)
df0 = df0.drop(df0[df0['Demographic_Category'].str.endswith('25-49_yrs')].index)
df0 = df0.drop(df0[df0['Demographic_Category'].str.endswith('65+_yrs')].index)

In [14]:
# Drop the rows with census counts of 0
df0 = df0.drop(df0[df0['census'] == 0].index)

In [15]:
# Drop the rows with up to date counts of 0
df0 = df0.drop(df0[df0['Up_to_date'] == 0].index)

In [16]:
# Drop the US territories as much  of the data is missing
df0 = df0.drop(df0[df0['Location'].isin(['AS', 'FM', 'GU', 'MH', 'MP', 'PR', 'PW', 'VI'])].index)

In [17]:
# Create and populate Gender column
df0['Gender'] = np.where(df0['Demographic_Category'].str.startswith('Female'), 'Female', 'Male')


In [18]:
# Create and populate Age Group column
df0['Age Group'] = np.where(df0['Demographic_Category'].str.endswith('<2yrs'), '< 2',
             np.where(df0['Demographic_Category'].str.endswith('2-4_yrs'), '2 to 4', 
             np.where(df0['Demographic_Category'].str.endswith('5-11_yrs'), '5 to 11',
             np.where(df0['Demographic_Category'].str.endswith('12-17_yrs'), '12 to 17',
             np.where(df0['Demographic_Category'].str.endswith('18-24_yrs'), '18 to 24',
             np.where(df0['Demographic_Category'].str.endswith('25-39_yrs'), '25 to 39',
             np.where(df0['Demographic_Category'].str.endswith('40-49_yrs'), '40 to 49',         
             np.where(df0['Demographic_Category'].str.endswith('50-64_yrs'), '50 to 64',
             np.where(df0['Demographic_Category'].str.endswith('65-74_yrs'), '65 to 74',        
             np.where(df0['Demographic_Category'].str.endswith('75+_yrs'), '75+', 'Unknown'))))))))))

In [19]:
# Displa ythe first 5 rows of the dataframe to verify the changes
df0.head()

,Date,Location,Demographic_Category,census,Up_to_date,Gender,Age Group
1,10/11/2023,AL,Female_Ages_2-4_yrs,88242,147,Female,2 to 4
2,10/11/2023,AR,Female_Ages_50-64_yrs,291030,44168,Female,50 to 64
5,10/11/2023,CA,Female_Ages_25-39_yrs,4290688,756614,Female,25 to 39
6,10/11/2023,CO,Female_Ages_25-39_yrs,640481,121887,Female,25 to 39
7,10/11/2023,CT,Female_Ages_<2yrs,34109,655,Female,< 2


In [20]:
# Check the record count and data types
df0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4285 entries, 1 to 12681
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Date                  4285 non-null   object
 1   Location              4285 non-null   object
 2   Demographic_Category  4285 non-null   object
 3   census                4285 non-null   int64 
 4   Up_to_date            4285 non-null   int64 
 5   Gender                4285 non-null   object
 6   Age Group             4285 non-null   object
dtypes: int64(2), object(5)
memory usage: 267.8+ KB


We're down to 4,285 records.

In [21]:
# Drop the 'Demographic_Category' column as it's no longer needed
df0=df0.drop('Demographic_Category', axis=1)

In [22]:
# Verify the changes
df0.head()

,Date,Location,census,Up_to_date,Gender,Age Group
1,10/11/2023,AL,88242,147,Female,2 to 4
2,10/11/2023,AR,291030,44168,Female,50 to 64
5,10/11/2023,CA,4290688,756614,Female,25 to 39
6,10/11/2023,CO,640481,121887,Female,25 to 39
7,10/11/2023,CT,34109,655,Female,< 2


In [23]:
# Rename the columns that don't match what we already have in Tableau
df0.rename(columns={'census': 'Census', 'Up_to_date': 'Up to Date', 'Location': 'State'}, inplace=True)

Renaming 'Location', to 'State' allows Tableau to automatically recognize the US state abbreviations as geographic data and allows us to easily create a map visualization.

In [24]:
# Load the state names file and join it to the dataframe
df1 = pd.read_csv("State Names.csv")

# Display the first 5 rows of the dataframe
df1.head()

,State,State Name
0,AL,Alabama
1,AK,Alaska
2,AZ,Arizona
3,AR,Arkansas
4,CA,California


In [25]:
# Merge the dataframes
df0 = df0.merge(df1, how='left', on='State')

In [26]:
# Verify the changes
df0.head()

,Date,State,Census,Up to Date,Gender,Age Group,State Name
0,10/11/2023,AL,88242,147,Female,2 to 4,Alabama
1,10/11/2023,AR,291030,44168,Female,50 to 64,Arkansas
2,10/11/2023,CA,4290688,756614,Female,25 to 39,California
3,10/11/2023,CO,640481,121887,Female,25 to 39,Colorado
4,10/11/2023,CT,34109,655,Female,< 2,Connecticut


I prefer to have state names to display in tooltips on a map in Tableau, so better to add them now with a few lines of code than have to manually add them later.

In [27]:
# Check to see how many records remain
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4285 entries, 0 to 4284
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        4285 non-null   object
 1   State       4285 non-null   object
 2   Census      4285 non-null   int64 
 3   Up to Date  4285 non-null   int64 
 4   Gender      4285 non-null   object
 5   Age Group   4285 non-null   object
 6   State Name  4285 non-null   object
dtypes: int64(2), object(5)
memory usage: 234.5+ KB


We still have 4,285 records. Nothing unexpected happened. 

In [28]:
# Save df0 to csv
df0.to_csv('COVID 19 Up to Date US.csv', index=False, encoding='utf-8')

Save cleaned data for visualization in Tableau.